# 06. RNN을 이용한 텍스트 생성(Text Generation using RNN) 

1. RNN을 이용하여 텍스트 생성하기 

1) 데이터에 대한 이해와 전처리 

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [4]:
text = """경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
vocab_size = len(tokenizer.word_index) + 1    # padding을 위한 0을 고려하여 +1 해줌

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 12


In [5]:
print(tokenizer.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [6]:
# 훈련 데이터
sequences = list()
for line in text.split('\n'):
  encoded = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

print('학습에 사용할 샘플의 개수: %d' % len(sequences))

학습에 사용할 샘플의 개수: 11


In [7]:
print(sequences)

[[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


- [2, 3]: [경마장에, 있는]
- [2, 3, 1]: [경마장에, 있는, 말이]
- training data는 맨 우측에 있는 단어에 대해서만 label로 분리해야 함 

In [8]:
max_len = max(len(l) for l in sequences)    # 모든 sample에서 길이가 가장 긴 sample의 길이 출력
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 6


In [9]:
sequences = pad_sequences(sequences, maxlen=max_len, padding = 'pre')

- pre_sequences(): 모든 sample에 대해 0을 사용하여 길이를 맞춰줌  
- padding = 'pre': 길이가 6보다 짧은 sample의 앞에 0으로 채움 

In [10]:
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [11]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [12]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [13]:
# 원-핫 인코딩
y = to_categorical(y, num_classes = vocab_size)

print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


2) 모델 설계하기 

- 다중 클래스 분류 문제는 활성 함수로 softmax, 손실 함수로 cross-entropy 사용 

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [15]:
embedding_dim = 10
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1/1 - 2s - loss: 2.4951 - accuracy: 0.0909 - 2s/epoch - 2s/step
Epoch 2/200
1/1 - 0s - loss: 2.4803 - accuracy: 0.0909 - 9ms/epoch - 9ms/step
Epoch 3/200
1/1 - 0s - loss: 2.4661 - accuracy: 0.1818 - 8ms/epoch - 8ms/step
Epoch 4/200
1/1 - 0s - loss: 2.4523 - accuracy: 0.1818 - 10ms/epoch - 10ms/step
Epoch 5/200
1/1 - 0s - loss: 2.4390 - accuracy: 0.2727 - 15ms/epoch - 15ms/step
Epoch 6/200
1/1 - 0s - loss: 2.4258 - accuracy: 0.3636 - 10ms/epoch - 10ms/step
Epoch 7/200
1/1 - 0s - loss: 2.4128 - accuracy: 0.4545 - 9ms/epoch - 9ms/step
Epoch 8/200
1/1 - 0s - loss: 2.3997 - accuracy: 0.4545 - 9ms/epoch - 9ms/step
Epoch 9/200
1/1 - 0s - loss: 2.3865 - accuracy: 0.5455 - 8ms/epoch - 8ms/step
Epoch 10/200
1/1 - 0s - loss: 2.3730 - accuracy: 0.5455 - 8ms/epoch - 8ms/step
Epoch 11/200
1/1 - 0s - loss: 2.3593 - accuracy: 0.5455 - 8ms/epoch - 8ms/step
Epoch 12/200
1/1 - 0s - loss: 2.3452 - accuracy: 0.5455 - 9ms/epoch - 9ms/step
Epoch 13/200
1/1 - 0s - loss: 2.3306 - accuracy: 0.5455 -

In [20]:
def sentence_generation(model, tokenizer, current_word, n):
  init_word = current_word
  sentence = ' '

  for _ in range(n):
    encoded = tokenizer.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=5, padding='pre')
    # 입력한 X(현재 단어)에 대해 Y를 예측하고 Y(예측한 단어)를 result에 저장
    result = model.predict(encoded, verbose=0)
    result = np.argmax(result, axis=1)

    for word, index in tokenizer.word_index.items():
      if index == result:
        break

    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word

  sentence = init_word + sentence
  
  return sentence

In [21]:
print(sentence_generation(model, tokenizer, '경마장에', 4))

경마장에  있는 말이 뛰고 있다


In [22]:
print(sentence_generation(model, tokenizer, '그의', 2))

그의  말이 법이다


In [23]:
print(sentence_generation(model, tokenizer, '가는', 5))

가는  말이 고와야 오는 말이 곱다


2. LSTM을 이용하여 텍스트 생성하기 

1) 데이터에 대한 이해와 전처리 

In [24]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [25]:
df = pd.read_csv('/content/drive/MyDrive/NLP_입문/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [26]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [27]:
print(df['headline'].isnull().values.any())

False


In [28]:
headline = []     # headline의 값들을 list로 저장
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [29]:
headline = [word for word in headline if word != "Unknown"]
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline)))

노이즈값 제거 후 샘플의 개수 : 1214


In [30]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

데이터 전처리(구두점 제거, 단어의 소문자화) 

In [31]:
def repreprocessing(raw_sentence):
  preproceseed_sentence = raw_sentence.encode("utf8").decode("ascii", 'ignore')
  return ''.join(word for word in preproceseed_sentence if word not in punctuation).lower()

preprocessed_headline = [repreprocessing(x) for x in headline]
preprocessed_headline[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [32]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_headline)
vocab_size =len(tokenizer.word_index) + 1

print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [33]:
sequences = list()

for sentence in preprocessed_headline:
  encoded = tokenizer.texts_to_sequences([sentence])[0]
  for i in range(1, len(encoded)):
    sequence = encoded[:i+1]
    sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

- [99, 269]: [former nfl]
- [99, 269, 371]: [former nfl cheerleaders]
- 하나의 단어를 예측하기 위해 이전에 등장한 단어들을 모두 참고해야 하므로, 하나의 문장을 나누는 것 

In [34]:
index_to_word = {}
for key, value in tokenizer.word_index.items():     # 인덱스를 단어로 바꾸기 위해 index_to_word 생성
 index_to_word[value] = key

print('빈도수 상위 582번 단어: {}'.format(index_to_word[582]))

빈도수 상위 582번 단어: offer


전체 샘플 길이 동일하게 하기 위해 padding 수행 

In [35]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 24


In [36]:
sequences = pad_sequences(sequences, maxlen = max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [38]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]

print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


In [39]:
print(y[:3])

[ 269  371 1115]


In [40]:
y = to_categorical(y, num_classes = vocab_size)

2) 모델 설계하기 

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [45]:
embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
244/244 - 14s - loss: 7.6389 - accuracy: 0.0310 - 14s/epoch - 58ms/step
Epoch 2/100
244/244 - 12s - loss: 7.1161 - accuracy: 0.0301 - 12s/epoch - 49ms/step
Epoch 3/100
244/244 - 10s - loss: 6.9802 - accuracy: 0.0331 - 10s/epoch - 41ms/step
Epoch 4/100
244/244 - 12s - loss: 6.8592 - accuracy: 0.0388 - 12s/epoch - 49ms/step
Epoch 5/100
244/244 - 12s - loss: 6.7190 - accuracy: 0.0411 - 12s/epoch - 48ms/step
Epoch 6/100
244/244 - 12s - loss: 6.5593 - accuracy: 0.0459 - 12s/epoch - 48ms/step
Epoch 7/100
244/244 - 12s - loss: 6.4666 - accuracy: 0.0513 - 12s/epoch - 49ms/step
Epoch 8/100
244/244 - 10s - loss: 6.2169 - accuracy: 0.0559 - 10s/epoch - 43ms/step
Epoch 9/100
244/244 - 12s - loss: 6.0373 - accuracy: 0.0597 - 12s/epoch - 48ms/step
Epoch 10/100
244/244 - 12s - loss: 5.8627 - accuracy: 0.0651 - 12s/epoch - 48ms/step
Epoch 11/100
244/244 - 12s - loss: 5.6919 - accuracy: 0.0674 - 12s/epoch - 48ms/step
Epoch 12/100
244/244 - 12s - loss: 5.5270 - accuracy: 0.0711 - 12s/epoch -

In [48]:
def sentence_generation(model, tokenizer, current_word, n):
  init_word = current_word
  sentence = ' '

  for _ in range(n):
    encoded = tokenizer.texts_to_sequences([current_word])[0]
    encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')
    result = model.predict(encoded, verbose=0)
    result = np.argmax(result, axis=1)

    for word, index in tokenizer.word_index.items():
      if index == result:
        break

    current_word = current_word + ' ' + word
    sentence = sentence + ' ' + word

  sentence = init_word + sentence
  
  return sentence

In [49]:
print(sentence_generation(model, tokenizer, 'i', 10))

i  disapprove of school vouchers can i still apply for them


In [51]:
print(sentence_generation(model, tokenizer, 'how', 10))

how  to make a crossword puzzle as is the people of
